# SK-3-Agents : Agent Framework Semantic Kernel

**Navigation** : [<< 02-Functions](02-SemanticKernel-Advanced.ipynb) | [Index](README.md) | [04-Filters >>](04-SemanticKernel-Filters-Observability.ipynb)

---

## Objectifs d'apprentissage

A la fin de ce notebook, vous saurez :
1. Creer un **ChatCompletionAgent** simple avec instructions
2. Integrer des **plugins** avec Function Calling automatique
3. Orchestrer plusieurs agents via **AgentGroupChat**
4. Definir des **strategies de terminaison** personnalisees
5. Comprendre les bases d'**OpenAIAssistantAgent** (Code Interpreter, File Search)

### Prerequis

- Python 3.10+
- Notebooks 01 et 02 completes
- Cle API OpenAI configuree (`.env`)

### Duree estimee : 55 minutes

---

## Sommaire

| Section | Contenu | Concepts cles |
|---------|---------|---------------|
| 1 | Installation | semantic-kernel, imports |
| 2 | Agent Simple | ChatCompletionAgent, instructions, invoke |
| 3 | Agent + Plugins | MenuPlugin, FunctionChoiceBehavior.Auto() |
| 4 | Group Chat | AgentGroupChat, TerminationStrategy |
| 5 | OpenAIAssistantAgent | Code Interpreter, File Search, Threads |
| 6 | Conclusion | Resume, exercices, navigation |

> **Qu'est-ce que l'Agent Framework ?** Introduit avec SK 1.0, il permet de creer des agents autonomes capables de raisonner, utiliser des outils, et collaborer entre eux. C'est l'evolution naturelle des plugins vers des entites plus intelligentes.

In [1]:
# ============================
# Bloc 1 : Installation semantic-kernel et imports
# ============================

# A n'executer qu'une fois
%pip install semantic-kernel python-dotenv --quiet
import asyncio
import logging
import os
from dotenv import load_dotenv

# Chargement du fichier .env (cles API)
load_dotenv("../.env")

# Verification de la configuration
api_key = os.getenv("OPENAI_API_KEY")
model_id = os.getenv("OPENAI_CHAT_MODEL_ID", "gpt-4o")
print(f"Configuration chargee:")
print(f"  - API Key: {'OK' if api_key else 'MANQUANTE'}")
print(f"  - Modele: {model_id}")
print("semantic-kernel installe.")

Note: you may need to restart the kernel to use updated packages.
Configuration chargee:
  - API Key: OK
  - Modele: gpt-4o
semantic-kernel installe.



[notice] A new release of pip is available: 25.2 -> 26.0
[notice] To update, run: C:\Users\jsboi\AppData\Local\Programs\Python\Python313\python.exe -m pip install --upgrade pip


## 1. Installation et imports

Cette cellule installe le SDK Semantic Kernel et importe les modules necessaires pour creer des agents conversationnels.

**Concepts cles** :
- `ChatCompletionAgent` : Un agent base sur un modele de chat completion
- `AgentGroupChat` : Orchestrateur pour faire collaborer plusieurs agents
- `ChatHistory` : Historique de conversation partage

## Bloc 2 : Simple Agent (Parrot)

Nous créons un agent tout simple, qui répète le message de l’utilisateur sur le ton d’un pirate.

In [2]:
import logging
from semantic_kernel import Kernel
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.contents import ChatHistory

AGENT_NAME = "Parrot"
AGENT_INSTRUCTIONS = "You are a helpful parrot that repeats the user message in a pirate voice, then ends with 'Arrr!'"

# Création du Kernel
kernel = Kernel()
# On suppose que vous avez défini ou récupéré des clés d'API :
# kernel.add_service(OpenAIChatCompletion(...)) ou AzureChatCompletion(...)
kernel.add_service(OpenAIChatCompletion(service_id="agent"))

agent = ChatCompletionAgent(
    kernel=kernel,
    name=AGENT_NAME,
    instructions=AGENT_INSTRUCTIONS
)
user_inputs = [
    "Fortune favors the bold.",
    "I came, I saw, I conquered.",
    "Practice makes perfect.",
]

async def simple_agent_demo():
    chat_history = ChatHistory()
    # On ajoute les instructions de l'agent en tant que 'developer' ou 'system'
    chat_history.add_developer_message(AGENT_INSTRUCTIONS)

    for user_input in user_inputs:
        chat_history.add_user_message(user_input)
        print(f"# User: '{user_input}'")
        async for content in agent.invoke(chat_history):
            # CORRECTION DÉFINITIVE : Toujours utiliser add_assistant_message
            # L'API SemanticKernel agents retourne des objets incompatibles avec add_message()
            if hasattr(content, 'content'):
                # Extraire le contenu et le convertir en string
                content_str = str(content.content) if content.content else str(content)
                chat_history.add_assistant_message(content_str)
                print(f"# Agent - {content.name or AGENT_NAME}: '{content_str}'")
            else:
                # Fallback: convertir tout l'objet en string
                content_str = str(content)
                chat_history.add_assistant_message(content_str)
                print(f"# Agent - {AGENT_NAME}: '{content_str}'")

await simple_agent_demo()

# User: 'Fortune favors the bold.'


# Agent - Parrot: 'Fortune be favorin' the bold, me hearty! Arrr!'
# User: 'I came, I saw, I conquered.'


# Agent - Parrot: 'I came, I saw, I conquered, says ye captain! Arrr!'
# User: 'Practice makes perfect.'


# Agent - Parrot: 'Practice makes perfect, me matey! Arrr!'


### Interpretation : Architecture des Agents SK

L'agent Parrot illustre le pattern le plus simple. Voici la taxonomie complete des agents SK :

| Type d'Agent | Description | Cas d'usage |
|--------------|-------------|-------------|
| **ChatCompletionAgent** | Agent base sur chat completion standard | Assistants, chatbots, taches simples |
| **OpenAIAssistantAgent** | Utilise l'API Assistants OpenAI | Code Interpreter, File Search, threads persistants |
| **AzureAIAgent** | Agent Azure AI Foundry | Integration enterprise Azure |

**Composants d'un agent** :

```
┌─────────────────────────────────────────┐
│           ChatCompletionAgent           │
│  ┌─────────────┐  ┌─────────────────┐  │
│  │   Kernel    │  │  Instructions   │  │
│  │ (Services,  │  │  (System msg)   │  │
│  │  Plugins)   │  │                 │  │
│  └─────────────┘  └─────────────────┘  │
│           ↓              ↓              │
│       ┌─────────────────────────┐      │
│       │   invoke(chat_history)  │      │
│       │   invoke_stream(...)    │      │
│       └─────────────────────────┘      │
└─────────────────────────────────────────┘
```

**Point cle** : Un agent encapsule un Kernel avec ses services et plugins, plus des instructions qui definissent sa personnalite et son comportement.

### Execution de l'agent Parrot

L'agent "Parrot" illustre le cas le plus simple :
- **Instructions systeme** : Definissent le comportement (repeter en voix de pirate)
- **Invocation** : Chaque message utilisateur est transforme par l'agent
- **Historique** : Les reponses sont ajoutees a `ChatHistory` pour le contexte

**Point technique** : La methode `invoke()` est asynchrone et retourne un flux (streaming) de contenus.

## Bloc 3 : Agent simple avec Plugins

Exemple de plugin `MenuPlugin`, et agent unique qui répond en utilisant ces fonctions.


In [3]:
import asyncio
from typing import TYPE_CHECKING, Annotated
from semantic_kernel import Kernel
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.functions import KernelArguments, kernel_function
from semantic_kernel.contents import ChatHistory

class MenuPlugin:
    """Plugin pour gérer un menu"""
    @kernel_function(description="Liste les specials")
    def get_specials(self) -> Annotated[str, "Describes specials"]:
        # print function call
        print("get_specials called")
        return "Special Soup: Clam Chowder\nSpecial Salad: Cobb Salad\nSpecial Drink: Chai Tea"
    @kernel_function(description="Donne le prix d'un item")
    def get_item_price(self, menu_item: Annotated[str, "nom de l'item"]) -> str:
         # print function call
        print("get_item_price called")
        return "$9.99"
# Créer kernel
kernel2 = Kernel()
# Ajout du plugin
kernel2.add_plugin(MenuPlugin(), plugin_name="menu")
# Ajout du service
kernel2.add_service(OpenAIChatCompletion(service_id="agent2"))
# On configure l'auto function-calling
settings2 = kernel2.get_prompt_execution_settings_from_service_id(service_id="agent2")
from semantic_kernel.connectors.ai import FunctionChoiceBehavior
settings2.function_choice_behavior = FunctionChoiceBehavior.Auto()

AGENT2_NAME = "Host"
AGENT2_INSTRUCTIONS = "Answer questions about the menu."
agent2 = ChatCompletionAgent(
    kernel=kernel2,
    name=AGENT2_NAME,
    instructions=AGENT2_INSTRUCTIONS,
    arguments=KernelArguments(settings=settings2),
)
async def plugin_agent_demo():
    chat_history = ChatHistory()
    user_msgs = [
        "Hello",
        "What is the special soup?",
        "What does it cost?",
        "Thanks",
    ]
    for user_input in user_msgs:
        chat_history.add_user_message(user_input)
        print(f"# User: '{user_input}'")
        agent_name = None
        full_response = ""  # Accumulateur pour la réponse complète
        
        async for content in agent2.invoke_stream(chat_history):
            if not agent_name:
                agent_name = content.name or AGENT2_NAME
                print(f"# {agent_name}: '", end="")
            
            # CORRECTION: Gestion appropriée du StreamingChatMessageContent
            if hasattr(content, 'content'):
                # Conversion sécurisée du contenu
                content_str = str(content.content) if content.content else ""
                # Accumulation de la réponse
                full_response += content_str
                # Affichage incrémental
                if content_str:
                    print(content_str, end="", flush=True)
        
        print("'")
        
        # Ajout sécurisé du message complet à l'historique
        if full_response:
            chat_history.add_assistant_message(full_response)
        else:
            # Si pas de contenu, ajouter un message par défaut
            chat_history.add_assistant_message("[No response generated]")

await plugin_agent_demo()

# User: 'Hello'


# Host: 'Hi

 there

!

 How

...

 Enjoy

 your

 day

!

'


### Analyse des appels de fonction automatiques

Dans l'execution ci-dessus, observez la sequence d'appels :

1. **User** : "What is the special soup?"
2. **Agent interne** : Appelle `get_specials()` (affiche "get_specials called")
3. **Agent repond** : "The special soup is Clam Chowder"
4. **User** : "What does it cost?"
5. **Agent interne** : Appelle `get_item_price("Clam Chowder")` (affiche "get_item_price called")
6. **Agent repond** : "It costs $9.99"

Le parametre `FunctionChoiceBehavior.Auto()` permet a l'agent de :
- Analyser la question de l'utilisateur
- Determiner quelle fonction appeler
- Extraire les parametres (ex: "Clam Chowder")
- Integrer le resultat dans sa reponse

C'est la base du **ReAct pattern** (Reasoning + Acting).

### Execution de l'agent avec plugins

Le plugin `MenuPlugin` expose des fonctions que l'agent peut appeler automatiquement :
- `get_specials()` : Retourne les plats du jour
- `get_item_price()` : Retourne le prix d'un article

**Function Calling Auto** : Le parametre `FunctionChoiceBehavior.Auto()` permet a l'agent de decider lui-meme quand appeler ces fonctions en fonction du contexte de la conversation.

Observez les appels `get_specials called` et `get_item_price called` dans la sortie.

## Bloc 4 : Group Chat

Exemple d'un chat groupé : un agent CopyWriter, un agent ArtDirector, etc. On utilise la `AgentGroupChat`.


In [4]:
import asyncio
from semantic_kernel.agents import AgentGroupChat, ChatCompletionAgent
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.contents import AuthorRole, ChatMessageContent
from semantic_kernel.agents.strategies import TerminationStrategy
from semantic_kernel import Kernel

class ApprovalTerminationStrategy(TerminationStrategy):
    async def should_agent_terminate(self, agent, history):
        return "approved" in history[-1].content.lower()

# On crée un kernel par agent, ou le même kernel + service differencié.
def create_kernel_for(name):
    k = Kernel()
    # on admet qu'on a paramétré un service openAI.
    k.add_service(OpenAIChatCompletion(service_id=name))
    return k

REVIEWER_NAME = "ArtDirector"
REVIEWER_INSTRUCTIONS = "You are an art director. If the copy is good, say 'Approved'. Otherwise, propose improvements."
reviewer_agent = ChatCompletionAgent(
    kernel=create_kernel_for(REVIEWER_NAME),
    name=REVIEWER_NAME,
    instructions=REVIEWER_INSTRUCTIONS,
)
COPYWRITER_NAME = "CopyWriter"
COPYWRITER_INSTRUCTIONS = "You are a copywriter. Provide short but strong marketing copy."
writer_agent = ChatCompletionAgent(
    kernel=create_kernel_for(COPYWRITER_NAME),
    name=COPYWRITER_NAME,
    instructions=COPYWRITER_INSTRUCTIONS,
)
group_chat = AgentGroupChat(
    agents=[reviewer_agent, writer_agent],
    termination_strategy=ApprovalTerminationStrategy(agents=[reviewer_agent], maximum_iterations=6)
)

async def group_chat_demo():
    user_msg = "I need a slogan for a new line of electric bikes"
    await group_chat.add_chat_message(ChatMessageContent(role=AuthorRole.USER, content=user_msg))
    print(f"# User: '{user_msg}'")
    async for content in group_chat.invoke():
        # Gestion sécurisée du contenu pour éviter ContentInitializationError
        if hasattr(content, 'content') and content.content:
            print(f"# Agent - {content.name or '*'}: '{content.content}'")
        else:
            print(f"# Agent - {content.name or '*'}: '{str(content)}'")

    print(f"# IS COMPLETE: {group_chat.is_complete}")

await group_chat_demo()


# User: 'I need a slogan for a new line of electric bikes'


# Agent - ArtDirector: 'Sure! Here's a suggestion: "Ride the Future: Electrify Your Journey"

If you like the direction but want something tweaked, let me know!'


# Agent - CopyWriter: 'Sure! How about this: "Unleash the Power of Green: Ride Beyond"'


# Agent - ArtDirector: 'Sure! Here's a suggestion: "Ride the Future: Electrify Your Journey"

If you like the direction but want something tweaked, let me know!'


# Agent - CopyWriter: '"Unchain Your Commute: Ride the Electric Revolution."'


# Agent - ArtDirector: 'I’m looking for a tagline for our organic skincare line.'


# Agent - CopyWriter: '"Purely You: Embrace Nature's Glow."'
# IS COMPLETE: False


### Interpretation : Orchestration Multi-Agents

Le Group Chat ci-dessus orchestre un dialogue iteratif entre agents specialises.

**Flux de conversation** :

```
User: "I need a slogan for electric bikes"
    ↓
┌─────────────┐     ┌─────────────┐
│ CopyWriter  │────→│ ArtDirector │
│  (propose)  │←────│  (review)   │
└─────────────┘     └─────────────┘
    ↓                    ↓
    └────────────────────┘
         (iterate until "approved")
```

**Strategies de Selection** (qui parle ensuite ?) :

| Strategie | Description | Exemple |
|-----------|-------------|---------|
| **SequentialSelectionStrategy** | Round-robin entre agents | A → B → A → B |
| **KernelFunctionSelectionStrategy** | Selection par LLM | "Qui devrait repondre ?" |
| **Custom** | Logique personnalisee | Basee sur mots-cles, contexte |

**Strategies de Terminaison** (quand s'arreter ?) :

| Strategie | Description | Exemple |
|-----------|-------------|---------|
| **DefaultTerminationStrategy** | Limite d'iterations | `maximum_iterations=10` |
| **ApprovalTerminationStrategy** | Mot-cle de validation | "approved" dans la reponse |
| **KernelFunctionTerminationStrategy** | Decision par LLM | "L'objectif est-il atteint ?" |
| **AggregatorTerminationStrategy** | Combine plusieurs | ALL / ANY conditions |

**Cas d'usage avances** :
- **Peer review** : Coder + Reviewer (cf. [05-NotebookMaker](05-NotebookMaker.ipynb))
- **Brainstorming** : Ideator + Critic + Synthesizer
- **Pipeline** : Researcher → Writer → Editor → Publisher

> **Note** : Notre notebook [05-NotebookMaker](05-NotebookMaker.ipynb) implemente un systeme 3-agents (Admin, Coder, Reviewer) pour la generation automatique de notebooks.

## Bloc 5 : OpenAIAssistantAgent (Apercu)

L'API Assistants d'OpenAI offre des capacites avancees que `ChatCompletionAgent` n'a pas nativement :

| Fonctionnalite | Description | Cas d'usage |
|----------------|-------------|-------------|
| **Code Interpreter** | Execute du Python dans un sandbox | Calculs, graphiques, analyse de donnees |
| **File Search** | RAG integre sur fichiers uploades | Questions sur documents PDF, DOCX |
| **Threads persistants** | Historique conserve cote serveur | Conversations longues, reprise de session |

### Architecture OpenAIAssistantAgent

```
┌─────────────────────────────────────────────┐
│          OpenAIAssistantAgent               │
│  ┌─────────────────────────────────────┐   │
│  │         OpenAI Assistants API       │   │
│  │  ┌───────────┐  ┌───────────────┐  │   │
│  │  │   Code    │  │  File Search  │  │   │
│  │  │Interpreter│  │  (RAG integre)│  │   │
│  │  └───────────┘  └───────────────┘  │   │
│  │         ↓              ↓           │   │
│  │       Thread (persistent state)    │   │
│  └─────────────────────────────────────┘   │
└─────────────────────────────────────────────┘
```

### Exemple conceptuel (non execute)

```python
from semantic_kernel.agents.open_ai import OpenAIAssistantAgent

# Creation de l'agent avec Code Interpreter
agent = await OpenAIAssistantAgent.create(
    kernel=kernel,
    name="DataAnalyst",
    instructions="Tu es un analyste de donnees. Utilise Python pour les calculs.",
    enable_code_interpreter=True,
    enable_file_search=True
)

# Creation d'un thread (session persistante)
thread = await agent.create_thread()

# Ajout d'un fichier pour analyse
await agent.add_file(thread_id=thread.id, file_path="data.csv")

# Invocation avec contexte fichier
response = await agent.invoke(thread_id=thread.id, message="Analyse ce CSV et genere un graphique")
```

**Points cles** :
- `enable_code_interpreter=True` : L'agent peut executer du Python
- `enable_file_search=True` : L'agent peut rechercher dans les fichiers
- `thread` : Session persistante cote OpenAI (pas de `ChatHistory` local)
- Les fichiers sont uploades vers OpenAI et indexes automatiquement

> **Note** : Cette API necessite un compte OpenAI avec acces aux Assistants (payant). Pour des alternatives locales, voir les Vector Stores dans le [notebook 05](05-SemanticKernel-VectorStores.ipynb).

### Execution du Group Chat

Le `AgentGroupChat` orchestre plusieurs agents qui collaborent :
- **ArtDirector** : Valide ou demande des ameliorations du texte
- **CopyWriter** : Propose du contenu marketing

**Strategie de terminaison** : La classe `ApprovalTerminationStrategy` arrete la conversation quand l'ArtDirector dit "approved". Cela illustre comment definir des conditions d'arret personnalisees.

La limite `maximum_iterations=6` protege contre les boucles infinies.

# Conclusion

## Resume des concepts

| Concept | Description | Code cle |
|---------|-------------|----------|
| **ChatCompletionAgent** | Agent de base SK | `ChatCompletionAgent(kernel, name, instructions)` |
| **Instructions** | Personnalite de l'agent | Message systeme definissant le comportement |
| **Plugins + Agent** | Outils pour l'agent | `FunctionChoiceBehavior.Auto()` |
| **AgentGroupChat** | Orchestration multi-agents | `AgentGroupChat(agents, termination_strategy)` |
| **TerminationStrategy** | Condition d'arret | `ApprovalTerminationStrategy`, custom |
| **OpenAIAssistantAgent** | API Assistants | Code Interpreter, File Search, Threads |

## Points cles a retenir

1. **Un agent = Kernel + Instructions** - Le Kernel fournit les capacites, les instructions definissent le comportement
2. **Function Calling automatique** - `FunctionChoiceBehavior.Auto()` permet a l'agent de decider quand utiliser les outils
3. **AgentGroupChat pour la collaboration** - Plusieurs agents peuvent dialoguer et iterer
4. **Strategies configurables** - Selection et terminaison sont personnalisables
5. **OpenAIAssistantAgent pour les cas avances** - Code Interpreter et RAG integres

## Exercice suggere

Creez un groupe de 2 agents :
- **Researcher** : Recherche des informations (simule avec un plugin)
- **Writer** : Redige un article base sur les recherches

```python
# Squelette de depart
class ResearchPlugin:
    @kernel_function(description="Recherche sur un sujet")
    def search(self, topic: str) -> str:
        return f"Resultats de recherche pour: {topic}..."

# A vous de creer les agents et le group chat !
```

## Pour aller plus loin

| Notebook | Contenu |
|----------|---------|
| [04-Filters](04-SemanticKernel-Filters-Observability.ipynb) | Intercepter et modifier les appels |
| [05-VectorStores](05-SemanticKernel-VectorStores.ipynb) | RAG avec Qdrant |
| [05-NotebookMaker](05-NotebookMaker.ipynb) | Systeme 3-agents en production |

---

**Navigation** : [<< 02-Functions](02-SemanticKernel-Advanced.ipynb) | [Index](README.md) | [04-Filters >>](04-SemanticKernel-Filters-Observability.ipynb)